In [ ]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC, run_GaBP_HARDWARE, run_GaBP_HARDWARE_WORSTCASE, run_GaBP_HARDWARE_BESTCASE, run_GaBP_SYNC_TEST
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [ ]:
num_nodes = 100

number_pes = 1

A, b = data_gen.get_1D_line_matrix(num_nodes, scaling=True, normalized=False)
# A, b = data_gen.get_2D_lattice_matrix(num_nodes, num_nodes)
graph = NetworkxGraph(A)
# graph.draw_graph()

convergence_threshold = 1*10**-5

In [ ]:
sum_of_iterations = 0
num_iterations = 1

for it in range(0,num_iterations):
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC(A, b, max_iter=1000, mae=False, convergence_threshold=convergence_threshold)
    sum_of_iterations += iteration
    final_mean = means[-1]
    final_std = stand_divs[-1]
    print(it)

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

print(final_mean)

In [ ]:
RESULTS = []
RESULT_WORST_CASE = []
RESULT_BEST_CASE = []

In [ ]:
for node_updates_per_pe in range(1, num_nodes+1):

    sum_of_iterations = 0
    num_iterations = 100

    node_updates_per_stream = number_pes*node_updates_per_pe

    for it in range(0,num_iterations):
        P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration, _ , _  = run_GaBP_HARDWARE(A, b, node_updates_per_pe=node_updates_per_pe, number_pes=number_pes, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=1, show=True)
        sum_of_iterations += iteration
        print(it)
        print("-----------")


    print(means[-1])
    print(f"AVE. WORSTCASE = {sum_of_iterations/num_iterations}")

    RESULTS.append(sum_of_iterations/num_iterations)

In [ ]:
for node_updates_per_pe in range(1, num_nodes+1):

    sum_of_iterations = 0
    num_iterations = 100

    node_updates_per_stream = number_pes*node_updates_per_pe

    for it in range(0,num_iterations):
        P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration, _ , _  = run_GaBP_HARDWARE_WORSTCASE(A, b, node_updates_per_stream=node_updates_per_stream, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=0.1)
        sum_of_iterations += iteration
        print(it)
        print("-----------")

    print(means[-1])
    print(f"AVE. WORSTCASE = {sum_of_iterations/num_iterations}")

    RESULT_WORST_CASE.append(sum_of_iterations/num_iterations)

In [ ]:
for node_updates_per_pe in range(1, num_nodes+1):

    sum_of_iterations = 0
    num_iterations = 100

    node_updates_per_stream = number_pes*node_updates_per_pe

    for it in range(0,num_iterations):
        P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_HARDWARE_BESTCASE(A, b, node_updates_per_stream=node_updates_per_stream, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=0.1)
        sum_of_iterations += iteration
        print(it)
        print("-----------")

    print(f"AVE. IDEAL = {sum_of_iterations/num_iterations}")

    RESULT_BEST_CASE.append(sum_of_iterations/num_iterations)

In [ ]:
import matplotlib.pyplot as plt

# Sample x and y data
x_values = [i for i in range(1, num_nodes+1)]

# Create the plot
plt.plot(x_values, RESULTS, label='Results')
plt.plot(x_values, RESULT_WORST_CASE, label='Worst Case')
plt.plot(x_values, RESULT_BEST_CASE, label='Best Case')

# Add labels and title
plt.xlabel('Number of Nodes')
plt.ylabel('Results')
plt.title('Performance Analysis')
plt.legend()  # Add legend based on labels provided in plot()

# Show the plot
plt.show()